In [1]:
import os
import numpy as np

from includes import config
from includes.utils import squad_dataset

import tensorflow as tf

In [2]:
def load_vocab(vocab_path):
    if os.path.exists(vocab_path):
        
        with open(vocab_path, mode="rb") as f:
            vocab = dict([
                (line.strip(), index)
                for index, line in enumerate(f.readlines())
            ])
        
        return vocab
    
    else:
        raise IOError("File %s not found.", vocab_path)

In [3]:
root_dir = os.getcwd()

In [4]:
train_data = squad_dataset(config.question_train, config.context_train, config.answer_train, root=root_dir)

In [5]:
LSTMCell = tf.contrib.rnn.BasicLSTMCell
DynamicRNN = tf.nn.dynamic_rnn

In [6]:
class Encoder:
    def __init__(self, hidden_size):
        self.hidden_size = hidden_size
        
    def encode(self, input, lengths):
        question, passage = input
        question_lengths, passage_lengths = lengths
        
        lstm_cell_question = tf.contrib.rnn.BasicLSTMCell(self.hidden_size, state_is_tuple=True)
        encoded_question, (q_rep, _) = DynamicRNN(
            lstm_cell_question,
            question,
            question_lengths,
            dtype=tf.float32
        )

        lstm_cell_passage = tf.contrib.rnn.BasicLSTMCell(self.hidden_size, state_is_tuple=True)
        encoded_passage, (p_rep, _) =  tf.nn.dynamic_rnn(
            lstm_cell_question,
            question,
            question_lengths,
            dtype=tf.float32
        )
            
        return encoded_question, q_rep, encoded_passage, p_rep

In [7]:
class Graph():
    def __init__(self):
        self.__init_nodes()
    
    def __init_nodes(self):
        self.encoder = Encoder(5)
        
        self.questions = tf.placeholder(tf.float32, [None, None, 10])
        self.passages = tf.placeholder(tf.float32, [None, None, 10])
        
        self.questions_length = tf.placeholder(tf.float32, [None])
        self.passages_length = tf.placeholder(tf.float32, [None])
        
        self.encoded_questions, \
        self.rep_questions, \
        self.encoded_passages, \
        self.rep_passages = self.encoder.encode(
            (self.questions, self.passages),
            (self.questions_length, self.passages_length)
        )
        
    def train(self):
        with tf.Session() as sess:
            tf.global_variables_initializer().run(session=sess)
            
            out = sess.run(
                [self.rep_questions, self.rep_passages],
                feed_dict={
                    self.questions: np.random.rand(5 * 10).reshape((1, 5, 10)).astype(dtype=np.float32),
                    self.passages: np.random.rand(5 * 10).reshape((1, 5, 10)).astype(dtype=np.float32),
                    self.questions_length: [5],
                    self.passages_length: [5]
                }
            )
        print out

In [8]:
graph = Graph()
graph.train()

[array([[ 0.03206288,  0.27108854, -0.8568663 , -0.5719821 ,  0.25359595]],
      dtype=float32), array([[ 0.03206288,  0.27108854, -0.8568663 , -0.5719821 ,  0.25359595]],
      dtype=float32)]
